<a href="https://colab.research.google.com/github/akhi07rx/CoLab-torrent-downloader/blob/main/CoLab_torrent_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install python3-libtorrent

In [ ]:
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent

In [ ]:
import libtorrent as lt
import time
from google.colab import drive
from tqdm import tqdm

drive.mount('/content/drive')

ses = lt.session()
ses.listen_on(6881, 6891)

save_path = '/content/drive/MyDrive/Torrent Downloads/'

params = {
    'save_path': save_path,
    'storage_mode': lt.storage_mode_t.storage_mode_sparse,
}

while True:
    print()
    link = input("Enter the magnet link (or 'exit' to quit): ")
    if link.lower() == 'exit':
        break

    if not re.match(magnet_link_pattern, link):
        print("Invalid magnet link. Please try again.")
        continue

    handle = lt.add_magnet_uri(ses, link, params)

    print("\nDownloading Metadata...")
    while not handle.has_metadata():
        time.sleep(1)
    print("Got Metadata, Starting Torrent Download...")

    torrent_info = handle.get_torrent_info()
    file_size = torrent_info.total_size()
    file_size_mb = file_size / (1024 ** 2)
    file_size_gb = file_size / (1024 ** 3)
    file_size_str = f"{file_size_mb:.2f} MB / {file_size_gb:.2f} GB"
    print(f"File size: {file_size_str}")

    progress_bar = tqdm(total=file_size, unit='B', unit_scale=True, leave=True)

    start_time = time.time()

    while handle.status().state != lt.torrent_status.seeding:
        s = handle.status()
        state_str = ['queued', 'checking', 'downloading metadata', 'downloading', 'finished', 'seeding', 'allocating']
        progress_bar.set_description(state_str[s.state])
        progress_bar.update(s.total_done - progress_bar.n)
        time.sleep(1)

    progress_bar.close()

    end_time = time.time()
    total_time_seconds = end_time - start_time
    total_time_minutes = total_time_seconds / 60

    num_files = torrent_info.num_files()

    print(f"\nFile name: {torrent_info.name()}")
    print(f"File size: {file_size_str}")
    print(f"Total Time: {total_time_seconds:.2f} seconds / {total_time_minutes:.2f} minutes")
    print(f"Number of files: {num_files}")
    print(f"Saved location: {save_path}")
